# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [ ]:
# Your code here
import requests
from bs4 import BeautifulSoup
import csv

def get_movie_reviews(movie_url, max_reviews=1000):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    reviews = []
    page_number = 1
    while len(reviews) < max_reviews:
        page_url = f'{movie_url}&page={page_number}'
        response = requests.get(page_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        review_divs = soup.find_all('div', class_='text show-more__control')
        if not review_divs:
            break  # No more reviews found
        for review_div in review_divs:
            if len(reviews) >= max_reviews:
                break
            reviews.append(review_div.text.strip())
        page_number += 1
    return reviews[:max_reviews]

def save_reviews_to_csv(movie_name, reviews):
    with open(f'{movie_name}_reviews.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Review'])
        for review in reviews:
            writer.writerow([review])

def main():
    # IMDb URL for the movie "Dune" released in 2023
    movie_url = 'https://www.imdb.com/title/tt1160419/reviews?ref_=tt_ql_3'
    movie_name = 'Dune'  # Movie name for file naming
    reviews = get_movie_reviews(movie_url)
    save_reviews_to_csv(movie_name, reviews)

if __name__ == "__main__":
    main()



# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re

nltk.download('stopwords')
nltk.download('wordnet')

# Initialize Spanish stopwords
stop_words_spanish = set(stopwords.words('spanish'))

# Initialize stemmer and lemmatizer for Spanish
stemmer_spanish = SnowballStemmer('spanish')
lemmatizer_spanish = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Define functions for data cleaning
Remove special characters and punctuations


In [ ]:
# Define functions for data cleaning
# Remove special characters and punctuations
def remove_noise(text):
    clean_text = re.sub(r'[^\w\sáéíóúü]', '', text)
    return clean_text

Remove Numbers

In [ ]:
def remove_numbers(text):
    clean_text = re.sub(r'\d+', '', text)
    return clean_text

Remove Stopwords

In [ ]:
def remove_stopwords(text):
    words = text.split()
    clean_words = [word for word in words if word.lower() not in stop_words_spanish]
    clean_text = ' '.join(clean_words)
    return clean_text


converting text to lowercase

In [ ]:
def lowercase(text):
    clean_text = text.lower()
    return clean_text

Performming Stemming

In [ ]:
def stemming(text):
    words = text.split()
    stemmed_words = [stemmer_spanish.stem(word) for word in words]
    clean_text = ' '.join(stemmed_words)
    return clean_text

Performing Lemmatization

In [ ]:
def lemmatization(text):
    words = text.split()
    lemmatized_words = [lemmatizer_spanish.lemmatize(word) for word in words]
    clean_text = ' '.join(lemmatized_words)
    return clean_text


In [ ]:
# Load the CSV file
df = pd.read_csv('Dune_reviews.csv')

# Apply each cleaning step and save the clean data in a new column
df['Clean_Text'] = df['Review'].apply(remove_noise)
df['Clean_Text'] = df['Clean_Text'].apply(remove_numbers)
df['Clean_Text'] = df['Clean_Text'].apply(remove_stopwords)
df['Clean_Text'] = df['Clean_Text'].apply(lowercase)
df['Clean_Text'] = df['Clean_Text'].apply(stemming)
df['Clean_Text'] = df['Clean_Text'].apply(lemmatization)

# Save the cleaned data to a new CSV file
df.to_csv('Dune_cleaned_reviews.csv', index=False)

print("Cleaned data saved to 'Dune_cleaned_reviews.csv'.")

Cleaned data saved to 'Dune_cleaned_reviews.csv'.


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

Parts of speech (POS) Tagging

In [ ]:
pip install stanfordnlp

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag, ne_chunk
from nltk.parse import CoreNLPParser
import spacy

# Load the cleaned text data
with open('/content/Dune_cleaned_reviews.csv', 'r', encoding='utf-8') as file:
    cleaned_text = file.read()

# Tokenize the text into sentences
sentences = sent_tokenize(cleaned_text)

# (1) Parts of Speech (POS) Tagging
noun_count = 0
verb_count = 0
adj_count = 0
adv_count = 0

for sentence in sentences:
    words = word_tokenize(sentence)
    tagged_words = pos_tag(words)
    for word, tag in tagged_words:
        if tag.startswith('N'):  # Noun
            noun_count += 1
        elif tag.startswith('V'):  # Verb
            verb_count += 1
        elif tag.startswith('J'):  # Adjective
            adj_count += 1
        elif tag.startswith('R'):  # Adverb
            adv_count += 1

print("Parts of Speech (POS) Tagging:")
print("Noun count:", noun_count)
print("Verb count:", verb_count)
print("Adjective count:", adj_count)
print("Adverb count:", adv_count)

# (2) Constituency Parsing (using NLTK)
grammar = r"""
    NP: {<DT|JJ|NN.*>+}          # Chunk sequences of DT, JJ, NN
    PP: {<IN><NP>}                # Chunk prepositions followed by NP
    VP: {<VB.*><NP|PP|CLAUSE>+$}  # Chunk verbs and their arguments
    CLAUSE: {<NP><VP>}            # Chunk NP, VP
"""
chunk_parser = nltk.RegexpParser(grammar)

print("\nConstituency Parsing Trees:")
for sentence in sentences[:1]:  # Print trees for the first sentence as an example
    words = word_tokenize(sentence)
    tagged_words = pos_tag(words)
    tree = chunk_parser.parse(tagged_words)
    print(tree)

# (3) Named Entity Recognition (NER) using spaCy
nlp = spacy.load("en_core_web_sm")

named_entities = []

for sentence in sentences:
    doc = nlp(sentence)
    named_entities.extend([(ent.text, ent.label_) for ent in doc.ents])

# Extract entities of interest
person_names = [entity[0] for entity in named_entities if entity[1] == 'PERSON']
organizations = [entity[0] for entity in named_entities if entity[1] == 'ORG']
locations = [entity[0] for entity in named_entities if entity[1] == 'GPE']
dates = [entity[0] for entity in named_entities if entity[1] == 'DATE']
product_names = [entity[0] for entity in named_entities if entity[1] == 'PRODUCT']

# Calculate counts of each entity
person_count = len(person_names)
organization_count = len(organizations)
location_count = len(locations)
product_count = len(product_names)
date_count = len(dates)

print("\nNamed Entity Recognition:")
print("Person names:", person_count)
print("Organizations:", organization_count)
print("Locations:", location_count)
print("Product names:", product_count)
print("Dates:", date_count)


Parts of Speech (POS) Tagging:
Noun count: 158105
Verb count: 94333
Adjective count: 56182
Adverb count: 40711

Constituency Parsing Trees:
(S
  (NP Review/NNP)
  ,/,
  (NP Clean_Text/NNP)
  ''/''
  (NP Denis/NNP Villeneuve/NNP)
  has/VBZ
  accomplished/VBN
  what/WP
  was/VBD
  considered/VBN
  (NP impossible/JJ)
  (PP for/IN (NP decades/NNS))
  ,/,
  to/TO
  write/VB
  and/CC
  direct/VB
  (NP a/DT faithful/JJ adaptation/NN)
  to/TO
  (NP the/DT fantastic/JJ)
  1965/CD
  (NP sci-fi/JJ novel/NN)
  (PP by/IN (NP Frank/NNP Herbert/NNP))
  ./.)

Named Entity Recognition:
Person names: 9987
Organizations: 3342
Locations: 965
Product names: 165
Dates: 3018


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
The assignment provided a comprehensive hands-on experience in natural language processing (NLP), covering data collection, cleaning, and various NLP tasks such as part-of-speech tagging, parsing, and named entity recognition. While challenging, especially in data collection and cleaning, the tasks were intellectually stimulating and rewarding, offering opportunities for exploration and problem-solving. The provided time to insufficient for me , depending on their prior experience and familiarity with NLP tools and techniques. Overall, the assignment offered valuable insights into NLP and practical application of skills in analyzing text data."